In [ ]:
# Deterministic Finsler scenario utilities
import numpy as np
import matplotlib.pyplot as plt

from fire_model.ca import FireEnv
from fire_model.finsler import FinslerFireModel
from fire_model.boundary import between_boundaries_mask
from fire_model.bo import RetardantDropBayesOpt


### Environment Set-Up


In [ ]:
# Grid / domain
nx = ny = 120
domain_km = 5

# Time discretisation (used for indexing time-varying wind and for FireState.t)
dt_s = 5.0

# Fire physics parameters
burn_time_s0 = 15 * 60         # seconds a cell remains burning after ignition
ros_mps = 0.2                  # baseline ROS (m/s)

# Retardant parameters
retardant_half_life_s = 2400.0 # seconds (e.g., 40 minutes)
retardant_k = 1.25             # attenuation strength exp(-k * retardant)

# Spatial maps
xs = np.linspace(-1.0, 1.0, nx)[:, None]
ys = np.linspace(-1.0, 1.0, ny)[None, :]
fuel = 0.7 + 0.9 * np.exp(-3.0 * (xs**2 + ys**2))
fuel = np.ones((nx, ny), dtype=float)
value = 1.0 + 3.0 * np.exp(-4.0 * ((xs - 0.2) ** 2 + (ys + 0.1) ** 2))
value = np.ones((nx, ny), dtype=float)

# Constant southerly wind pushing the fire northward
wind=np.zeros((nx, ny, 2), dtype=float)
wind[..., 0] = 0.0
wind[..., 1] = 15.0

env = FireEnv(
    grid_size=(nx, ny),
    domain_km=domain_km,
    wind=wind,
    fuel=fuel,
    value=value,
    dt_s=dt_s,
    burn_time_s0=burn_time_s0,
    retardant_half_life_s=retardant_half_life_s,
    retardant_k=retardant_k,
    ros_mps=ros_mps,
    wind_coeff=0.6,
    diag=True,
    avoid_burning_drop=True,
    avoid_drop_p_threshold=0.05,
)

finsler_model = FinslerFireModel(env, k_wind=0.3, w_ref=5.0, clamp=(0.1, 5.0))


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 4), constrained_layout=True)

im0 = ax[0].imshow(fuel.T, origin="lower", aspect="equal")
ax[0].set_title("Fuel")
ax[0].set_xlabel("x cell"); ax[0].set_ylabel("y cell")
fig.colorbar(im0, ax=ax[0], fraction=0.046)

im1 = ax[1].imshow(value.T, origin="lower", aspect="equal")
ax[1].set_title("Value")
ax[1].set_xlabel("x cell"); ax[1].set_ylabel("y cell")
fig.colorbar(im1, ax=ax[1], fraction=0.046)

plt.show()


### Ignition and Search Domain


In [ ]:
ignition_center = (nx // 2, ny // 2)
ignition_radius_km = 0.01
time_to_first_drop = 100.0
time_to_second_drop = 240.0
search_space_time = (time_to_second_drop - time_to_first_drop) // 2

init_firestate_finsler = finsler_model.simulate_from_ignition(
    center=ignition_center,
    radius_km=ignition_radius_km,
    T=time_to_first_drop,
)

finsler_model.plot_firestate(init_firestate_finsler, title="Initial Finsler FireState")

init_boundary = finsler_model.extract_fire_boundary(
    init_firestate_finsler,
    K=200,
    p_boundary=0.25,
    field="affected",
    anchor="max_x",
    ccw=True,
)

finsler_model.plot_fire_boundary(
    init_firestate_finsler,
    init_boundary,
    field="affected",
    title="Fire Boundary at Time of First Drop",
)

second_firestate_finsler = finsler_model.simulate_from_firestate(
    init_firestate=init_firestate_finsler,
    T=time_to_second_drop - time_to_first_drop,
)

second_boundary = finsler_model.extract_fire_boundary(
    second_firestate_finsler,
    K=200,
    p_boundary=0.25,
    field="affected",
    anchor="max_x",
    ccw=True,
)

finsler_model.plot_fire_boundary(
    second_firestate_finsler,
    second_boundary,
    field="affected",
    title="Fire Boundary at Time of Second Drop (No Intervention)",
)

between_boundaries = between_boundaries_mask(
    init_boundary.xy,
    second_boundary.xy,
    finsler_model.env.grid_size,
)
finsler_model.plot_search_domain(
    between_boundaries,
    title="Search Domain for First Drone Drop",
)

constrained_firestate = finsler_model.simulate_from_firestate(
    init_firestate=init_firestate_finsler,
    T=search_space_time,
)

constrained_boundary = finsler_model.extract_fire_boundary(
    constrained_firestate,
    K=200,
    p_boundary=0.25,
    field="affected",
    anchor="max_x",
    ccw=True,
)

finsler_model.plot_fire_boundary(
    constrained_firestate,
    constrained_boundary,
    field="affected",
    title="Fire Boundary at Constrained Search Timepoint",
)

constrained_search_domain = between_boundaries_mask(
    init_boundary.xy,
    constrained_boundary.xy,
    finsler_model.env.grid_size,
)
finsler_model.plot_search_domain(
    constrained_search_domain,
    title="Search Domain for First Drone Drop (Constrained)",
)


### Bayesian Optimisation Set-Up


In [ ]:
constrained_search_time = (time_to_second_drop - time_to_first_drop) * 2 // 3

drop_opt = RetardantDropBayesOpt(
    fire_model=finsler_model,
    init_firestate=init_firestate_finsler,
    n_drones=4,
    evolution_time_s=time_to_second_drop - time_to_first_drop,
    search_grid_evolution_time_s=constrained_search_time,
    n_sims=1,
    fire_boundary_probability=0.25,
)


In [ ]:
best_theta, best_params, best_y, (X_feats, y_arr), y_nexts, y_bests_arr = drop_opt.run_heuristic_search(
    n_evals=200,
    heuristic_random_frac=0.0,
    print_every=20,
)


In [ ]:
drop_opt.plot_evolved_firestate(
    theta=best_theta,
    n_sims=1,
)


### Bayesian Optimisation (Random Mask Initialisation)


In [ ]:
best_theta, best_params, best_value, (X, y), y_nexts, y_bests = drop_opt.run_bayes_opt(
    n_init=50,
    n_iters=150,
    n_candidates=2**12,
    xi=0.01,
    verbose=True,
    init_strategy="random_mask",
    candidate_strategy="qmc",
    candidate_qmc="sobol",
    print_every=10,
)


In [ ]:
drop_opt.plot_evolved_firestate(
    theta=best_theta,
    n_sims=1,
)


### Bayesian Optimisation (Heuristic + Random Initialisation)


In [ ]:
search_mask, coords = drop_opt.setup_search_grid(K=300, boundary_field="affected")
best_theta, best_params, best_value, (X, y), y_nexts, y_bests = drop_opt.run_bayes_opt(
    n_init=50,
    n_iters=150,
    n_candidates=2**12,
    xi=0.01,
    verbose=True,
    init_strategy="heuristic",
    init_heuristic_random_frac=0.3,
    print_every=10,
    candidate_strategy="qmc",
    candidate_qmc="sobol",
)


In [ ]:
drop_opt.plot_evolved_firestate(
    theta=best_theta,
    n_sims=1,
)


### Heuristic + Local Exploitation


In [ ]:
search_mask, coords = drop_opt.setup_search_grid(K=300, boundary_field="affected")
best_theta, best_params, best_value, (X, y), y_nexts, y_bests = drop_opt.run_bayes_opt(
    n_init=50,
    n_iters=150,
    n_candidates=2**12,
    xi=0.01,
    verbose=True,
    init_strategy="heuristic",
    init_heuristic_random_frac=0.3,
    print_every=10,
    candidate_strategy="mixed",
    candidate_qmc="sobol",
    candidate_local_frac=0.6,
    candidate_local_top_k=5,
    candidate_local_sigma_cells=4.0,
    candidate_local_sigma_phi_rad=np.deg2rad(20),
)


In [ ]:
drop_opt.plot_evolved_firestate(
    theta=best_theta,
    n_sims=1,
)


In [ ]:
search_mask, coords = drop_opt.setup_search_grid(K=300, boundary_field="affected")
best_theta, best_params, best_value, (X, y), y_nexts, y_bests = drop_opt.run_bayes_opt(
    n_init=80,
    n_iters=160,
    n_candidates=2**12,
    xi=0.03,
    verbose=True,
    init_strategy="heuristic",
    init_heuristic_random_frac=0.3,
    print_every=20,
    candidate_strategy="mixed",
    candidate_qmc="sobol",
    candidate_local_frac=0.65,
    candidate_local_top_k=5,
    candidate_local_sigma_cells=4.0,
    candidate_local_sigma_phi_rad=np.deg2rad(20),
)
